# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium

In [3]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [4]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os
import glob
import torch
import pandas_ta as ta
import shutil

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder

In [6]:
import gymnasium as gym
from gym import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [7]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import SubprocVecEnv

In [8]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [9]:
import plotly.express as px
import plotly.graph_objects as go

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Используем GPU
    print("CUDA доступна. Работаем на GPU.")
else:
    device = torch.device("cpu")   # Используем CPU
    print("CUDA не доступна. Работаем на CPU.")


CUDA доступна. Работаем на GPU.


# Open and Preprocessing Data

- Теперь это Дефолтная модель
- Изменили window_length тепер оно работает и выставили его на 48
- Все показатели нормализуются, кроме RSI 
- нормализация StandartScaler
- Reset после потери половины стартового баланса и после смены токена
- Добавляем разделение на test и train режимы в среду
- Лучше всего работает модель Sac, далее идет модель A2C, которая тренируется быстрее, TD3 хуево 
- Уменьшение количества фич, не улучшает результат
- Пробуем новую функцию награды

### Settings

In [11]:
window_length = 48
nb_steps = 80000
scaler = StandardScaler()

model_num = 13
data_num = 1

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed= seed)

### Delete Data

In [12]:
def manage_directory(folder_path):
    # Проверяем, существует ли папка
    if os.path.exists(folder_path):
        # Если папка существует, удаляем все её содержимое
        shutil.rmtree(folder_path)
        print(f"Папка {folder_path} существовала и её содержимое было удалено.")
    
    # Создаем папку заново
    os.makedirs(folder_path)
    print(f"Папка {folder_path} была создана.")

In [13]:
folder_path = Path["model_dir"](model_num)
manage_directory(folder_path)

Папка d:\PythonScripts\trader_game\models\model_v13 была создана.


### Preprocessing Data

In [14]:
class CreateDataSet:
    def __init__(self, model_num, data_num, scaler, folder_path):
        self.model_num = model_num
        self.data_num = data_num
        self.scaler = scaler
        self.folder_path = folder_path
        self.numerical_features = [
            "Close", 'Open', 'High', 'Low', 'Average', 'Change', 'Volume', 'Volume Change',
            'EMA', 'SMA', 'RSI', 'MACD', 'BB_upper', 'BB_middle', 'BB_lower'
        ]


    def load_and_combine_data(self):
        csv_files = [f for f in os.listdir(self.folder_path) if f.endswith('.csv')]
        dataframes = []

        for csv_file in csv_files:
            file_path = os.path.join(self.folder_path, csv_file)
            df = pd.read_csv(file_path)
            df = df.iloc[1:].reset_index(drop=True)
            asset_id = os.path.splitext(csv_file)[0].split("_")[0]
            df['Asset_ID'] = asset_id
            dataframes.append(df)

        combined_df = pd.concat(dataframes, ignore_index=True)
        combined_df.fillna(method='ffill', inplace=True)
        combined_df.fillna(method='bfill', inplace=True)
        combined_df['Date'] = pd.to_datetime(combined_df['Date'])
        combined_df.sort_values(by=['Asset_ID', 'Date'], inplace=True)
        combined_df.reset_index(drop=True, inplace=True)
        return combined_df


    def add_technical_indicators(self, df):
        def indicators(group):
            group['EMA'] = ta.ema(group['Close'], length=14)
            group['SMA'] = ta.sma(group['Close'], length=14)
            group['RSI'] = ta.rsi(group['Close'], length=14)
            group['MACD'] = ta.macd(group['Close'])['MACD_12_26_9']
            bbands = ta.bbands(group['Close'], length=20)
            group['BB_upper'] = bbands['BBU_20_2.0']
            group['BB_middle'] = bbands['BBM_20_2.0']
            group['BB_lower'] = bbands['BBL_20_2.0']
            return group

        return df.groupby('Asset_ID', group_keys=False).apply(indicators)


    def add_time_features(self, df):
        df['Hour'] = df['Date'].dt.hour
        df['Day'] = df['Date'].dt.dayofweek
        df['Month'] = df['Date'].dt.month

        df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
        df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)

        df['Day_sin'] = np.sin(2 * np.pi * df['Day'] / 7)
        df['Day_cos'] = np.cos(2 * np.pi * df['Day'] / 7)

        df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
        df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
        return df


    def encode_and_scale(self, df):
        label_encoder = LabelEncoder()
        df['Asset_ID_encoded'] = label_encoder.fit_transform(df['Asset_ID'])
        df['Close_orig'] = df['Close']

        def scale_group(group):
            if self.scaler == "log":
                group[self.numerical_features] = np.log(group[self.numerical_features] + 1e-6)
            else:
                group[self.numerical_features] = self.scaler.fit_transform(group[self.numerical_features])
            return group

        return df.groupby('Asset_ID', group_keys=False).apply(scale_group)


    def replace_zeros(self, df):
        def replace_zeros_with_mean(group):
            for column in self.numerical_features + ["Close_orig"]:
                group[column].replace(0, np.nan, inplace=True)
                group[column].interpolate(method='linear', inplace=True)
                group[column].fillna(method='bfill', inplace=True)
                group[column].fillna(method='ffill', inplace=True)
            return group

        return df.groupby('Asset_ID', group_keys=False).apply(replace_zeros_with_mean)


    def run(self):
        df = self.load_and_combine_data()
        df = self.add_technical_indicators(df)
        df = self.add_time_features(df)
        df = self.encode_and_scale(df)
        df = self.replace_zeros(df)

        df.drop(columns=['Date', 'Asset_ID'], inplace=True)
        df.to_csv(Path["dataset"](model_num, data_num), index= False)
        del df  

        print(f"Dataset saved to {Path['dataset'](model_num, data_num)}")

In [15]:
create_data_set = CreateDataSet(model_num= model_num, data_num= data_num, scaler= scaler, folder_path= Path["raw"])
create_data_set.run()

C:\Users\McHomak\AppData\Local\Temp\ipykernel_5508\826862639.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='ffill', inplace=True)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_5508\826862639.py:27: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='bfill', inplace=True)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_5508\826862639.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('Asset_ID', group_keys=False).apply(indicators)
C:\Users\McHoma

Dataset saved to d:\PythonScripts\trader_game\models\model_v13\dataset_13_1.csv


### Open Data

In [16]:
df = pd.read_csv(Path["dataset"](model_num, data_num))
df.shape

(103954, 26)

In [17]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [18]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [19]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4388
23    4388
18    4388
0     4387
5     4387
16    4384
15    4384
6     4379
25    4372
19    4371
4     4367
14    4342
13    4334
20    4314
3     4313
8     4309
22    4219
9     4207
21    4093
24    3602
11    3589
10    3582
17    3120
7     2957
2     2931
12    1847
Name: count, dtype: int64

In [20]:
def split_by_asset_ids(df: pd.DataFrame, test_asset_ids: list):
    test_df = df[df['Asset_ID_encoded'].isin(test_asset_ids)]
    train_df = df[~df['Asset_ID_encoded'].isin(test_asset_ids)]
    return train_df, test_df

In [21]:
test_asset_ids = [21, 22, 23, 24, 25]
train_df, test_df = split_by_asset_ids(df = df, test_asset_ids = test_asset_ids)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (83280, 26)
Testing data shape: (20674, 26)


# Game Rule

In [22]:
class TradingEnv(gym.Env):
    """
    Environment for training an agent to trade on the exchange using a continuous action space.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df, mode):
        super(TradingEnv, self).__init__()

        # Save data and initialize parameters
        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length
        self.mode = mode # test or train

        # Find indices where a new asset starts
        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        # Define action space: Continuous action between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        # Define observation space
        num_features = len(self.df.columns) - 1 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        # Initialize trading parameters
        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'] - 1)
        print(self.now_token)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'shares_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {asset_ids[0]:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = seed, options=None, reset_hist=False):
        super().reset(seed= seed)
        self.logger.info("Environment reset")

        # Existing reset logic
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token not in self.asset_start_indices:
            if self.mode == "train":
                self.now_token = list(self.asset_start_indices.keys())[0]  # Loop back to the first asset
            else:
                return None, None

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        # Reset hist only if reset_hist is True
        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'shares_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]
        # obs = np.concatenate([
        #     frame.values,
        #     [self.shares_held],
        #     [self.balance],
        #     [self.net_worth]
        # ])
        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.prev_token = self.now_token

        terminated = False
        truncated = False

        self.current_step += 1  # Move to the next time step

        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                self.now_token -=1
                terminated = True


        if self.prev_net_worth != 0:
            net_worth_change = self.net_worth - self.prev_net_worth
            percent_change = (net_worth_change) / self.prev_net_worth * 100

            if percent_change > 0:
                self.reward += percent_change
            else:
                self.reward -= percent_change * 2  

            if self.net_worth < self.initial_balance * 0.5: # Только во время тренировки штрафуем за проеб половины баланса
                if self.mode == "train":
                    self.reward += -50  
                    self.logger.info("Net worth dropped below 50% of initial balance.")
                    terminated = True
        else:
            self.logger.info("prev_net_worth == 0")

        self.prev_net_worth = self.net_worth

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["shares_held"].append(self.shares_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)

        return obs, self.reward, terminated, truncated, info


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.shares_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)
        else:
            # Hold
            self.reward += -1
            pass  

        self.net_worth = self.balance + self.shares_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.shares_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.shares_held:
            shares_to_sell = self.shares_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.shares_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("No shares to sell.")


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.shares_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [23]:
train_env = TradingEnv(train_df, mode = "train")

{0: 0, 1: 4387, 2: 8775, 3: 11706, 4: 16019, 5: 20386, 6: 24773, 7: 29152, 8: 32109, 9: 36418, 10: 40625, 11: 44207, 12: 47796, 13: 49643, 14: 53977, 15: 58319, 16: 62703, 17: 67087, 18: 70207, 19: 74595, 20: 78966}
-1


In [24]:
test_env = TradingEnv(test_df, mode = "test")

{21: 0, 22: 4093, 23: 8312, 24: 12700, 25: 16302}
20


In [25]:
# check_env(train_env)

# Train Model

## Settings

In [26]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=512,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/",  # Directory for TensorBoard logs
    seed= seed,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [27]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model"](model_num, data_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [28]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        
    # callback=eval_callback   
    progress_bar= True,
)

Logging to ./sac_tensorboard/SAC_1


Output()

In [29]:
model.save(Path["model"](model_num, data_num))

In [30]:
train_hist = train_env.hist
train_hist_df = pd.DataFrame(train_hist)
print(len(train_hist["action"]))

80000


In [31]:
unique_values = train_hist_df['token'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

# Visualisaton

## Functions

In [32]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()

In [33]:
def plot_reward_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['reward'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['reward'], mode='lines', name='Reward'))

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Reward = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Reward and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Reward')

    # Показать график
    fig.show()

In [34]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()

In [35]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()

In [36]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()

In [37]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()

## Plots

In [38]:
token = 20
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [39]:
train_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,452.087899,999.452635,9568979,0,0.000057,0.109473,0.547912,0,0.000000
1,50,520.493723,1012.168317,8399238,0,0.000059,1.272265,-0.122243,1169741,68.405824
2,51,147.197833,1009.897166,14794542,0,0.000058,0.448769,0.717196,1169741,68.405824
3,52,89.093562,992.174437,15810793,0,0.000057,3.509809,0.394736,1169741,68.405824
4,53,812.739770,980.841024,2978037,0,0.000056,2.284561,-0.811645,14002497,792.052032
...,...,...,...,...,...,...,...,...,...,...
79995,43030,1497.363390,4858.968432,34927944,10,0.000096,2.374619,0.599540,4735370101,802256.608866
79996,43031,256.266657,4967.227680,47403989,10,0.000099,2.228029,0.828855,4735370101,802256.608866
79997,43032,1221.854791,4294.688813,36061048,10,0.000085,27.079043,-0.239282,4746713042,803222.197000
79998,43033,3629.662092,4352.248118,8317537,10,0.000087,1.340244,-0.769348,4774456553,805630.004301


In [40]:
train_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,80000.000000,8.000000e+04,8.000000e+04
mean,33636.576638,6.746922e+02,2540.641156,1.075957e+08,8.067713,9.151988e+00,-0.466503,0.258269,7.551577e+09,2.297686e+05
std,21727.834573,2.235243e+03,5107.158589,4.287505e+08,5.570349,2.178900e+01,4.015971,0.679805,2.553172e+10,5.940431e+05
min,49.000000,4.382720e-09,453.960365,0.000000e+00,0.000000,8.500000e-08,-49.390377,-1.000000,0.000000e+00,0.000000e+00
25%,13552.000000,3.276146e-01,815.868133,3.600000e+01,3.000000,1.438617e-04,-3.947144,-0.350087,1.978000e+04,3.097888e+04
50%,32637.500000,7.194528e+01,1052.297078,2.489500e+03,8.000000,5.532943e-02,0.095329,0.490558,1.351935e+06,8.085090e+04
75%,50543.250000,6.831178e+02,1823.814326,9.999392e+06,13.000000,1.644848e+00,1.049331,0.884838,1.307509e+09,1.718963e+05
max,80549.000000,5.530148e+04,69530.887654,6.598183e+09,20.000000,1.065761e+02,188.989386,0.999984,1.881535e+11,3.895460e+06


# Test Model

## Test

In [41]:
model = SAC.load(Path["model"](model_num, data_num))

In [42]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist
        if info is None:
            break

In [43]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20433


In [44]:
test_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,34.976489,999.035941,42272185,21,0.000023,0.192812,0.965024,0,0.000000
1,50,1.731690,974.569406,43766318,21,0.000022,4.898029,0.950490,0,0.000000
2,51,0.078690,974.742820,43840596,21,0.000022,0.017794,0.954566,0,0.000000
3,52,0.017019,968.166669,43843386,21,0.000022,1.349310,0.783833,0,0.000000
4,53,0.007226,974.918541,43843826,21,0.000022,0.697387,0.575781,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
20428,20669,447.166196,1116.225704,1,25,669.059508,-3.973117,-0.960184,27,8761.077884
20429,20670,447.166196,1118.779270,1,25,671.613074,-4.771232,-0.975088,27,8761.077884
20430,20671,447.166196,1115.851252,1,25,668.685056,-4.476569,-0.940785,27,8761.077884
20431,20672,447.166196,1113.058055,1,25,665.891859,-4.499360,-0.961078,27,8761.077884


In [45]:
unique_values = test_hist_df['token'].unique()
unique_values

array([21, 22, 23, 24, 25], dtype=int64)

## Plots

In [46]:
token = 23
plot_token_data(df = test_hist_df, token = token)
plot_reward_data(df = test_hist_df, token = token)
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [47]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - hist_df: DataFrame containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall averages for each column
    averages = report_df.mean(numeric_only=True)
    averages['Asset_ID'] = 'Average'  # Mark row as average

    # Append averages row to the DataFrame using pd.concat
    report_df = pd.concat([report_df, pd.DataFrame([averages])], ignore_index=True)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

- Asset_ID: Уникальный идентификатор актива (из столбца token), для которого рассчитываются метрики.

- Total Profit: Общий финансовый результат (прибыль или убыток) по данному активу. Рассчитывается как разница между конечной чистой стоимостью (net_worth) и начальным балансом (initial_balance).

- ROI (%): Доходность инвестиций (Return on Investment) в процентах. Показывает процентный прирост (или убыток) от начальной суммы баланса.

- Sharpe Ratio: Коэффициент Шарпа. Оценивает отношение доходности к риску (волатильности). Чем выше коэффициент Шарпа, тем лучше риск-корректированная доходность стратегии.

- Max Drawdown (%): Максимальная просадка в процентах. Это максимальное снижение стоимости актива от его исторического максимума. Отражает риски стратегии, связанные с падением стоимости.

- Win Rate (%): Процент прибыльных сделок. Это отношение количества прибыльных сделок к общему количеству сделок по активу, умноженное на 100.

- Buy-and-Hold Profit: Прибыль при стратегии "купить и держать". Показывает, сколько можно было бы заработать, если просто купить актив в начале и держать его до конца периода тестирования.

- Buy-and-Hold ROI (%): Доходность при стратегии "купить и держать". Процентный прирост от начальной цены актива, если его просто держать до конца периода.

- Ideal Profit: Идеальная прибыль. Это гипотетическая максимальная прибыль, которую можно было бы получить, если бы купили актив по минимальной цене и продали по максимальной цене за период.

- Ideal ROI (%): Идеальная доходность. Процентный прирост при идеальной стратегии, где покупка происходит по минимальной цене, а продажа — по максимальной.

- Asset Price Change (%): Изменение цены актива в процентах за период. Это процентное изменение цены от начальной до конечной за период тестирования.

In [48]:
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_5508\3864715686.py:85: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [49]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,21,-259.514089,-25.951409,-0.090722,-61.403365,45.686032,-279.352882,-27.935288,1596.459700,159.645970,-27.935288
1,22,5280.410915,528.041091,0.470469,-63.681878,49.340686,253802.131121,25380.213112,293446.958188,29344.695819,25380.213112
2,23,244.223360,24.422336,0.209365,-76.560270,48.894009,-152.651940,-15.265194,9635.856272,963.585627,-15.265194
3,24,-506.613153,-50.661315,-0.064559,-66.049071,49.437254,594.470161,59.447016,2103.489180,210.348918,59.447016
4,25,113.580079,11.358008,0.113707,-52.144033,49.479528,233.832793,23.383279,2790.670479,279.067048,23.383279
5,Average,974.417422,97.441742,0.127652,-63.967723,48.567502,50839.685851,5083.968585,61914.686764,6191.468676,5083.968585


In [50]:
test_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,20433.000000,2.043300e+04,20433.000000,2.043300e+04,20433.000000,20433.000000,20433.000000,20433.000000,2.043300e+04,20433.000000
mean,10360.856702,6.533141e+02,1245.254793,3.883458e+06,22.997015,78.111095,-2.491878,0.041853,2.721508e+08,32061.161962
std,5965.411689,1.101919e+03,1082.304586,1.104289e+07,1.419961,154.761422,3.198470,0.740527,6.683013e+08,33355.973783
min,49.000000,5.233771e-07,394.247990,0.000000e+00,21.000000,0.000011,-5.000000,-1.000000,0.000000e+00,0.000000
25%,5205.000000,1.944507e+00,619.680285,1.000000e+00,22.000000,0.008798,-4.976914,-0.762066,1.432000e+03,7532.565119
50%,10361.000000,3.139405e+02,866.926408,5.600000e+01,23.000000,0.272110,-4.064843,0.137813,5.737200e+04,21093.109009
75%,15517.000000,6.550390e+02,1333.777780,3.123000e+03,24.000000,8.408479,0.029637,0.792145,1.071189e+07,45897.284245
max,20673.000000,7.800940e+03,8043.404589,4.515070e+07,25.000000,674.820251,45.583020,0.999314,3.034675e+09,169698.049929


In [52]:
report_df.to_csv(Path["reports"](model_num, data_num), index= False)

In [53]:
overall_metrics

{'Total Profit': 5846.504534572054,
 'ROI (%)': 116.93009069144107,
 'Sharpe Ratio': 0.12765199235906544,
 'Win Rate (%)': 48.56750178918744,
 'Buy-and-Hold Profit': 305038.1151049347,
 'Buy-and-Hold ROI (%)': 6100.762302098695}